In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE42986"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE42986"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE42986.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE42986.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE42986.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome profiling in human primary mitochondrial respiratory chain disease"
!Series_summary	"Primary mitochondrial respiratory chain (RC) diseases are heterogeneous in etiology and manifestations but collectively impair cellular energy metabolism.  To identify a common cellular response to RC disease, systems biology level transcriptome investigations were performed in human RC disease skeletal muscle and fibroblasts.  Global transcriptional and post-transcriptional dysregulation in a tissue-specific fashion was identified across diverse RC complex and genetic etiologies.  RC disease muscle was characterized by decreased transcription of cytosolic ribosomal proteins to reduce energy-intensive anabolic processes, increased transcription of mitochondrial ribosomal proteins, shortened 5'-UTRs to improve translational efficiency, and stabilization of 3'-UTRs containing AU-rich elements.  These same modifications in a reversed direction typified 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re
from typing import Dict, Any, List, Callable, Optional
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptome profiling using
# Affymetrix Human Exon 1.0ST microarray, which likely contains gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# Examining the data more carefully, this dataset seems to be about mitochondrial respiratory chain disease,
# not epilepsy. The trait variable in our context is Epilepsy, but we don't see information about epilepsy 
# in the sample characteristics.
# Looking at the background information and sample characteristics dictionary, this doesn't appear to be
# an epilepsy-related dataset, so trait data is not available.
trait_row = None

# Age
# Row 3 contains age information
age_row = 3

# Gender
# Row 2 contains gender information
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert trait status to binary.
    Not used in this dataset as trait data is not available.
    """
    return None

def convert_age(value):
    """
    Convert age to continuous numeric value.
    """
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Handle "not obtained" case
    if value.lower() == "not obtained":
        return None
    
    # Try to extract numeric value
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """
    Convert gender to binary.
    0 = Female
    1 = Male
    """
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Use the validate_and_save_cohort_info function to save initial filtering results
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we don't have trait data available for this cohort
# and we'll skip the clinical feature extraction step
if trait_row is not None:
    # This code won't execute as trait_row is None
    # But would be used if trait data were available
    pass


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE42986/GSE42986_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE42986/GSE42986_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (20788, 53)
First 20 gene/probe identifiers:
['100009676_at', '10000_at', '10001_at', '10002_at', '100033416_at', '100033422_at', '100033423_at', '100033424_at', '100033425_at', '100033426_at', '100033428_at', '100033431_at', '100033434_at', '100033436_at', '100033438_at', '100033439_at', '100033444_at', '100033800_at', '100033806_at', '100033819_at']


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# Looking at the format of the identifiers: '100009676_at', '10000_at', etc.
# These appear to be Affymetrix probe IDs (with _at suffix) 
# rather than standard human gene symbols

# Affymetrix probe IDs need to be mapped to standard gene symbols
# for proper biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, 'ID' appears to be the probe ID and 'Description' contains gene names
# Display more samples from the Description column to better understand the format
print("\nSample of Description column (first 5 rows):")
if 'Description' in gene_annotation.columns:
    for i in range(min(5, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['Description'].iloc[i]}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Gene_ID', 'ORF', 'Symbol', 'Chromosome', 'RefSeq_ID', 'Num_Probes', 'Full_Name']
{'ID': ['1_at', '2_at', '9_at', '10_at', '12_at'], 'Gene_ID': ['1', '2', '9', '10', '12'], 'ORF': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3'], 'Symbol': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3'], 'Chromosome': ['19', '12', '8', '8', '14'], 'RefSeq_ID': ['NM_130786;NP_570602', 'NM_000014;NP_000005', 'NM_000662;NM_001160170;NM_001160171;NM_001160172;NM_001160173;NM_001160174;NM_001160175;NM_001160176;NM_001160179;NP_000653;NP_001153642;NP_001153643;NP_001153644;NP_001153645;NP_001153646;NP_001153647;NP_001153648;NP_001153651', 'NM_000015;NP_000006', 'NM_001085;NP_001076'], 'Num_Probes': [47.0, 167.0, 74.0, 20.0, 56.0], 'Full_Name': ['alpha-1-B glycoprotein', 'alpha-2-macroglobulin', 'N-acetyltransferase 1 (arylamine N-acetyltransferase)', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'serpin peptidase inhibitor, clade A (alpha-1 antip

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# Based on the previews:
# - 'ID' in gene_annotation contains probe IDs (like '1_at', '2_at', etc.)
# - 'Symbol' contains gene symbols (like 'A1BG', 'A2M', etc.)
print("\nMapping probe IDs to gene symbols...")
prob_col = 'ID'  # Column containing the probe identifiers
gene_col = 'Symbol'  # Column containing the gene symbols

# 2. Get a gene mapping dataframe by extracting the two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping (first 5 rows):")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nGene expression data after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(list(gene_data.index[:10]))

# Normalize gene symbols to ensure consistency across the dataset
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene expression data after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(list(gene_data.index[:10]))

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nSaved processed gene expression data to: {out_gene_data_file}")



Mapping probe IDs to gene symbols...
Gene mapping dataframe shape: (20788, 2)
Sample of gene mapping (first 5 rows):
{'ID': ['1_at', '2_at', '9_at', '10_at', '12_at'], 'Gene': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3']}

Gene expression data after mapping: (19870, 53)
First 10 gene symbols after mapping:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']



Gene expression data after normalization: (19636, 53)
First 10 normalized gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1', 'AAAS']



Saved processed gene expression data to: ../../output/preprocess/Epilepsy/gene_data/GSE42986.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (19636, 53)
Gene data shape after normalization: (19636, 53)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE42986.csv
No trait data (Epilepsy) available in this dataset based on previous analysis.
Cannot proceed with data linking due to missing trait or gene data.
Abnormality detected in the cohort: GSE42986. Preprocessing failed.
